### import Neccessary Libraries

In [12]:
import os 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time 
import torchvision.models as models
import matplotlib.pyplot as plt
import optuna

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Load Data

In [14]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2,contrast=0.2),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [15]:
dataset_path ="./dataset"

dataset = datasets.ImageFolder(root=dataset_path,transform=image_transforms)
len(dataset)

2300

In [16]:
dataset.classes

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [17]:
num_classes = len(dataset.classes)
num_classes

6

### Training Dataset

In [18]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

train_size,val_size

(1725, 575)

In [19]:
from torch.utils.data import random_split

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

In [20]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=32,shuffle=True)

### Model Traininng And Hyperparameter Tunning

In [21]:
class CarClassifierResNet(nn.Module):
    def __init__(self,num_classes,dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')

        for param in self.model.parameters():
            param.requires_grad = False
        
        for param in self.model.layer4.parameters():
            param.requires_grad = True

        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features,num_classes)
        )

    def forward(self,x):
        x = self.model(x)
        return x

In [23]:
def objective(trial):
    lr = trial.suggest_float('lr',1e-5,1e-2,log=True)
    dropout_rate = trial.suggest_float('dropout_rate',0.2,0.7)

    model = CarClassifierResNet(num_classes=num_classes,dropout_rate=dropout_rate).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p : p.requires_grad,model.parameters()),lr=lr)

    epochs = 3
    start = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num,(images , labels) in enumerate(train_loader):
            images,labels = images.to(device),labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            output = model(images)
            loss = criterion(output,labels)
            
            # Backward pass & optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation
        model.eval()
        total = 0
        correct = 0

        with torch.no_grad():
            for images , labels in val_loader:
                images,labels = images.to(device),labels.to(device)
                output = model(images)
                _,predicted = torch.max(output.data,1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            accuracy = 100 * correct / total

            trial.report(accuracy,epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    end = time.time()
    print(f"Execution time : {end - start} Seconds")
    
    return accuracy

In [24]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=20)

[I 2025-10-15 20:44:30,615] A new study created in memory with name: no-name-4f1afa74-9b72-4c59-bf69-ba04d6829d81
[I 2025-10-15 20:50:32,022] Trial 0 finished with value: 75.82608695652173 and parameters: {'lr': 0.00029120879734147957, 'dropout_rate': 0.5433250025826173}. Best is trial 0 with value: 75.82608695652173.


Execution time : 360.32098507881165 Seconds


[I 2025-10-15 20:56:20,999] Trial 1 finished with value: 56.69565217391305 and parameters: {'lr': 1.6848046243616204e-05, 'dropout_rate': 0.344279045287073}. Best is trial 0 with value: 75.82608695652173.


Execution time : 348.2177426815033 Seconds


[I 2025-10-15 21:02:18,185] Trial 2 finished with value: 75.47826086956522 and parameters: {'lr': 0.007262433986549524, 'dropout_rate': 0.5312178647770802}. Best is trial 0 with value: 75.82608695652173.


...

[I 2025-10-15 21:15:50,333] Trial 5 pruned. 
[I 2025-10-15 21:19:15,443] Trial 6 finished with value: 79.1304347826087 and parameters: {'lr': 0.0010891945890169682, 'dropout_rate': 0.688162983060687}. Best is trial 6 with value: 79.1304347826087.


Execution time : 204.64887189865112 Seconds


[I 2025-10-15 21:20:30,912] Trial 7 pruned. 
[I 2025-10-15 21:23:16,081] Trial 8 pruned. 
[I 2025-10-15 21:24:46,511] Trial 9 pruned. 
[I 2025-10-15 21:28:52,910] Trial 10 finished with value: 80.34782608695652 and parameters: {'lr': 0.0008652442999786002, 'dropout_rate': 0.6889213517602946}. Best is trial 10 with value: 80.34782608695652.


Execution time : 245.80264139175415 Seconds


[I 2025-10-15 21:30:20,889] Trial 11 pruned. 
[I 2025-10-15 21:33:20,560] Trial 12 pruned. 
[I 2025-10-15 21:34:47,225] Trial 13 pruned. 
[I 2025-10-15 21:37:46,233] Trial 14 pruned. 
[I 2025-10-15 21:39:15,441] Trial 15 pruned. 
[I 2025-10-15 21:42:09,884] Trial 16 pruned. 
[I 2025-10-15 21:43:38,141] Trial 17 pruned. 
[I 2025-10-15 21:45:05,219] Trial 18 pruned. 
[I 2025-10-15 21:46:32,116] Trial 19 pruned. 


### Print Best Parameter

In [25]:
study.best_params

{'lr': 0.0008652442999786002, 'dropout_rate': 0.6889213517602946}